<a href="https://colab.research.google.com/github/dine1717/END2/blob/Session14/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation

In [1]:
!pip install simpletransformers

In [2]:
!wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
!tar -xvf data/paws_wiki_labeled_final.tar.gz -C data

--2021-08-13 18:50:12--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 74.125.137.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘data/paws_wiki_labeled_final.tar.gz.2’

paws_wiki_labeled_f 100%[===================>]   4.47M  --.-KB/s    in 0.02s   

2021-08-13 18:50:12 (257 MB/s) - ‘data/paws_wiki_labeled_final.tar.gz.2’ saved [4687157/4687157]

final/test.tsv
final/
final/train.tsv
final/dev.tsv


In [3]:
import warnings

import pandas as pd


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [4]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

2021-08-13 18:50:17.383637: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [5]:
# Google Data
train_df = pd.read_csv("/content/data/final/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("/content/data/final/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

# # MSRP Data
# train_df = pd.concat(
#     [
#         train_df,
#         load_data("data/msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )
# eval_df = pd.concat(
#     [
#         eval_df,
#         load_data("data/msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )

# # Quora Data

# # The Quora Dataset is not separated into train/test, so we do it manually the first time.
# df = load_data(
#     "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
# )
# q_train, q_test = train_test_split(df)

# q_train.to_csv("data/quora_train.tsv", sep="\t")
# q_test.to_csv("data/quora_test.tsv", sep="\t")

# # The code block above only needs to be run once.
# # After that, the two lines below are sufficient to load the Quora dataset.

# # q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
# # q_test = pd.read_csv("data/quora_test.tsv", sep="\t")

# train_df = pd.concat([train_df, q_train])
# eval_df = pd.concat([eval_df, q_test])

# train_df = train_df[["prefix", "input_text", "target_text"]]
# eval_df = eval_df[["prefix", "input_text", "target_text"]]

# train_df = train_df.dropna()
# eval_df = eval_df.dropna()

# train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
# train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

# eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
# eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df.head(),eval_df.head())

                                          input_text  ...      prefix
1  The NBA season of 1975 -- 76 was the 30th seas...  ...  paraphrase
3  When comparable rates of flow can be maintaine...  ...  paraphrase
4  It is the seat of Zerendi District in Akmola R...  ...  paraphrase
5  William Henry Henry Harman was born on 17 Febr...  ...  paraphrase
7  With a discrete amount of probabilities Formul...  ...  paraphrase

[5 rows x 3 columns]                                           input_text  ...      prefix
1  They were there to enjoy us and they were ther...  ...  paraphrase
2  After the end of the war in June 1902 , Higgin...  ...  paraphrase
3  From the merger of the Four Rivers Council and...  ...  paraphrase
4  The group toured extensively and became famous...  ...  paraphrase
5  Kathy and her husband Pete Beale ( Peter Dean ...  ...  paraphrase

[5 rows x 3 columns]


In [6]:
    import torch
    torch.cuda.empty_cache()

In [7]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [ ]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 32
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)

model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )

INFO:filelock:Lock 140515205169104 acquired on /root/.cache/huggingface/transformers/3f12fb71b844fcb7d591fdd4e55027da90d7b5dd6aa5430ad00ec6d76585f26c.58d5dda9f4e9f44e980adb867b66d9e0cbe3e0c05360cefe3cd86f5db4fff042.lock


Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

INFO:filelock:Lock 140515205169104 released on /root/.cache/huggingface/transformers/3f12fb71b844fcb7d591fdd4e55027da90d7b5dd6aa5430ad00ec6d76585f26c.58d5dda9f4e9f44e980adb867b66d9e0cbe3e0c05360cefe3cd86f5db4fff042.lock
INFO:filelock:Lock 140515205168784 acquired on /root/.cache/huggingface/transformers/d065edfe6954baf0b989a2063b26eb07e8c4d0b19354b5c74af9a51f5518df6e.6ca4df1a6ec59aa763989ceec10dff41dde19f0f0824b9f5d3fcd35a8abffdb2.lock


Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

INFO:filelock:Lock 140515205168784 released on /root/.cache/huggingface/transformers/d065edfe6954baf0b989a2063b26eb07e8c4d0b19354b5c74af9a51f5518df6e.6ca4df1a6ec59aa763989ceec10dff41dde19f0f0824b9f5d3fcd35a8abffdb2.lock
INFO:filelock:Lock 140515209438736 acquired on /root/.cache/huggingface/transformers/0d6fc8b2ef1860c1f8f0baff4b021e3426cc7d11b153f98e563b799603ee2f25.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


...

  0%|          | 0/3539 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.4918663708476333}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/111 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


RuntimeError: ignored

In [8]:
model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs/checkpoint-2729-epoch-1"
)


In [9]:
eval_df_sample = eval_df.sample(frac=0.02, random_state=42)

In [10]:
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df_sample["prefix"].tolist(), eval_df_sample["input_text"].tolist())
]
truth = eval_df_sample["target_text"].tolist()

In [11]:
eval_df_sample.shape

(71, 3)

In [12]:
preds = model.predict(to_predict)

Generating outputs:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [14]:
to_predict[:5]

['paraphrase: The medals were presented by Carlo Croce , IOC member , New Zealand and Barbara Kendall , President of World Sailing .',
 'paraphrase: The BBC College of Journalism was opened as an e-learning course series in June 2005 , with Vin Ray as Executive Editor . Its first Director was Kevin Marsh .',
 "paraphrase: The original team was produced by the writer Sal Buscema and artist Roy Thomas in `` The Avengers '' # 71 ( December 1969 ) .",
 'paraphrase: Katuwana Divisional Secretariat is a Divisional Secretariat of Southern Province , of Hambantota District , Sri Lanka .',
 'paraphrase: Mats Wilander defeats Anders Järryd , 6 -- 4 , 3 -- 6 , 7 -- 5 .']

In [13]:

preds[:5]

[['Paraphrase : The medals were presented by Carlo Croce, IOC member, New Zealand and Barbara Kendall, President of World Sailing.',
  'Paraphrase : The medals were presented by Carlo Croce, IOC member, New Zealand and Barbara Kendall, President of World Sailing.',
  'Paraphrase : The medals were presented by Carlo Croce, IOC member, New Zealand and Barbara Kendall, President of World Sailing.'],
 ['Paraphrase : The BBC College of Journalism was opened in June 2005 as an e-learning course series with Vin Ray as Executive Editor, and its first director was Kevin Marsh.',
  'Paraphrase : The BBC College of Journalism was opened in June 2005 as an e-learning course series with Vin Ray as Executive Editor, and its first director was Kevin Marsh.',
  'Paraphrase : The BBC College of Journalism was opened in June 2005 as an e-learning course series with Vin Ray as Executive Editor, and its first director was Kevin Marsh.'],
 ["Paraphrase : The original team was produced by writer Sal Buscema

In [15]:
# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(to_predict):
        f.write(str(text).split(':', 1)[1].strip() + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )

In [16]:
import random

for i, text in enumerate(to_predict[:128]):
    print(f'Text  > {str(text).split(":", 1)[1].strip()}')
    print(f'Pred  < {preds[i][0]}')
    print(f'Truth = {truth[i]}')
    print()

Text  > The medals were presented by Carlo Croce , IOC member , New Zealand and Barbara Kendall , President of World Sailing .
Pred  < Paraphrase : The medals were presented by Carlo Croce, IOC member, New Zealand and Barbara Kendall, President of World Sailing.
Truth = The medals have been presented by Carlo Croce , IOC - member , New Zealand , and Barbara Kendall , president of World Sailing .

Text  > The BBC College of Journalism was opened as an e-learning course series in June 2005 , with Vin Ray as Executive Editor . Its first Director was Kevin Marsh .
Pred  < Paraphrase : The BBC College of Journalism was opened in June 2005 as an e-learning course series with Vin Ray as Executive Editor, and its first director was Kevin Marsh.
Truth = In June 2005 , the BBC College of Journalism was opened as an e-learning series with Vin Ray as Executive Editor , whose first director was Kevin Marsh .

Text  > The original team was produced by the writer Sal Buscema and artist Roy Thomas in 